# Library Setting

In [1]:
from tqdm import tqdm
tqdm.pandas()

import numpy as np
import pandas as pd
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import rc
rc('font', family='AppleGothic')
plt.rcParams['axes.unicode_minus'] = False

<br></br>

# Data

## Data Load

In [2]:
train_df = pd.read_csv('./data/train.csv')
test_df  = pd.read_csv('./data/test.csv')

In [3]:
train_df.shape, test_df.shape

((84406, 20), (17289, 19))

In [4]:
train_df.head()

,ID,월,요일,시간,소관경찰서,소관지역,사건발생거리,강수량(mm),강설량(mm),적설량(cm),풍향,안개,짙은안개,번개,진눈깨비,서리,연기/연무,눈날림,범죄발생지,TARGET
0,TRAIN_00000,9,화요일,10,137,8.0,2.611124,0.000,0.0,0.0,245.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,차도,2
1,TRAIN_00001,11,화요일,6,438,13.0,3.209093,0.000,0.0,0.0,200.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,차도,0
2,TRAIN_00002,8,일요일,6,1729,47.0,1.619597,0.000,0.0,0.0,40.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,인도,1
3,TRAIN_00003,5,월요일,6,2337,53.0,1.921615,11.375,0.0,0.0,225.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,주거지,1
4,TRAIN_00004,9,일요일,11,1439,41.0,1.789721,0.000,0.0,0.0,255.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,주유소,2


<br>

## Resetting Columns Type

In [5]:
def type_resetting(data):
    d = data.copy()
    target_feature = ['TARGET']
    cat_features = ['ID','월','요일','시간','소관경찰서','소관지역','범죄발생지']
    num_features = [col for col in d.columns if col not in cat_features]
    for col in cat_features:
        d[col] = d[col].astype(str)
    for col in num_features:
        d[col] = d[col].astype(float)
    if 'TARGET' in d.columns:
        d['TARGET'] = d['TARGET'].astype(str)
        d = d.rename(columns={'TARGET':'target'})
    return d

In [6]:
unuse_features = ['ID']

train_df2 = type_resetting(train_df)
test_df2  = type_resetting(test_df)

train_df2.drop(columns=unuse_features,inplace=True)
test_df2 .drop(columns=unuse_features,inplace=True)

<br></br>

# New Features

In [7]:
def new_features(data):
    d = data.copy()
    
    # (1) 강수,강설,적설여부 : 강수량이 0이면 강수여부=0, 강수
    d['강수여부'] = np.where(d['강수량(mm)']==0,0,1)
    d['강설여부'] = np.where(d['강설량(mm)']==0,0,1)
    d['적설여부'] = np.where(d['적설량(cm)']==0,0,1)
    
    # (2) 주말여부, 계절
    d['주말여부'] = np.where((d['요일']=='토요일')|(d['요일']=='일요일'),1,0)
    d['계절'] = ['겨울' if month in [12,1,2] else
                '봄'  if month in [3,4,5]  else
                '여름' if month in [6,7,8] else
                '가을' if month in [9,10,11] else
                'Unknown' for month in d['월'].astype(int)]
    
    # (3) 강수 grouping
    # - 참조 : https://namu.wiki/w/%EA%B0%95%EC%9A%B0%EB%9F%89#s-2
    d['강수량_체감'] = ['0'  if rainfall==0   else
                     '1'  if rainfall<=1   else
                     '2'  if rainfall<=2.5 else
                     '3'  if rainfall<=5.0 else
                     '4'  if rainfall<=10  else
                     '5'  if rainfall<=15  else
                     '6'  if rainfall<=20  else
                     '7'  if rainfall<=30  else
                     '8'  if rainfall<=40  else
                     '9'  if rainfall<=50  else
                     '10' if rainfall<=70  else
                     '11' if rainfall<=110 else
                     '12' for rainfall in d['강수량(mm)']]
    
    # (4) 풍향
    d['풍향_방위'] = ['N'  if (direction>=337.5) or  (direction< 22.5) else
                    'NE' if (direction>= 22.5) and (direction< 67.5) else
                    'E'  if (direction>= 67.5) and (direction<112.5) else
                    'SE' if (direction>=112.5) and (direction<157.5) else
                    'S'  if (direction>=157.5) and (direction<202.5) else
                    'SW' if (direction>=202.5) and (direction<247.5) else
                    'W'  if (direction>=247.5) and (direction<292.5) else
                    'NW' if (direction>=292.5) and (direction<337.5) else
                    'Unknown' for direction in d['풍향']]
    
    return d

In [8]:
train_df3 = new_features(train_df2)
test_df3  = new_features(test_df2)

In [9]:
target_feature = 'target'
cat_features = train_df3.columns[train_df3.dtypes==object]
cat_features = list(set(cat_features)-set([target_feature]))
num_features = [col for col in train_df3.columns if col not in cat_features+[target_feature]]

In [10]:
train_df3.head()

,월,요일,시간,소관경찰서,소관지역,사건발생거리,강수량(mm),강설량(mm),적설량(cm),풍향,안개,짙은안개,번개,진눈깨비,서리,연기/연무,눈날림,범죄발생지,target,강수여부,강설여부,적설여부,주말여부,계절,강수량_체감,풍향_방위
0,9,화요일,10,137,8.0,2.611124,0.000,0.0,0.0,245.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,차도,2.0,0,0,0,0,가을,0,SW
1,11,화요일,6,438,13.0,3.209093,0.000,0.0,0.0,200.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,차도,0.0,0,0,0,0,가을,0,S
2,8,일요일,6,1729,47.0,1.619597,0.000,0.0,0.0,40.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,인도,1.0,0,0,0,1,여름,0,NE
3,5,월요일,6,2337,53.0,1.921615,11.375,0.0,0.0,225.0,1.0,1.0,0.0,0.0,0.0,0.0,0.0,주거지,1.0,1,0,0,0,봄,5,SW
4,9,일요일,11,1439,41.0,1.789721,0.000,0.0,0.0,255.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,주유소,2.0,0,0,0,1,가을,0,W


<br></br>

# EDA

In [11]:
# i=0
# for col in num_features:
#     i+=1
#     print('\n({}/{}) {}'.format(i,len(num_features),col))
#     plt.figure(figsize=(15,7))
#     sns.violinplot(x=train_df3['target'],y=train_df3[col])
#     plt.show()

<br></br>

# Add the Interaction Term

In [12]:
import warnings
from tqdm import trange
def add_interaction_term(data,num_features):
    warnings.simplefilter(action='ignore', category=pd.errors.PerformanceWarning)
    
    d = data.copy()
    for i in range(len(num_features)):
        for j in range(len(num_features)):
            if i>j:
                col_i = num_features[i]
                col_j = num_features[j]
                d[f'{col_i}*{col_j}'] = d[col_i]*d[col_j]
                
    return d

In [13]:
train_df4 = add_interaction_term(train_df3,num_features)
test_df4  = add_interaction_term(test_df3 ,num_features)

In [14]:
target_feature = 'target'
cat_features = train_df4.columns[train_df4.dtypes==object]
cat_features = list(set(cat_features)-set([target_feature]))
num_features = [col for col in train_df4.columns if col not in cat_features+[target_feature]]

<br></br>

# Feature Selection

In [15]:
alpha = 0.05

<br>

## Numerical Features

In [16]:
# import statsmodels.api as sm
from statsmodels.formula.api import ols
from statsmodels.stats.anova import anova_lm

In [17]:
def log_offset(x):
    if min(x)>0:
        offset = 0
    elif min(x)==0:
        offset = 1e-3
    else:
        offset = min(x)+1e-3
        print('minimum = {:.3f}'.format(min(x)))
    return np.log(x+offset)

In [18]:
# (1) ANOVA를 해서 p-value가 0.05보다 높은 것들 확인
pvalue_list = []
for col in tqdm(num_features):
    d = train_df4[[col,'target']].rename(columns={col:'feature'})
    
    model = ols(f'feature ~ C(target)',data=d).fit()
    pvalue = anova_lm(model).values[0][-1]
    pvalue_list.append([col,pvalue])
    
pvalue_df = pd.DataFrame(pvalue_list,columns=['feature','pvalue'])\
    .sort_values('pvalue',ascending=False)
# pvalue_df[pvalue_df.pvalue>=alpha].round(4)

100%|██████████| 136/136 [00:41<00:00,  3.29it/s]


In [21]:
# (2) (1)에서 유의하지않은 feature들은 log적용 후에도 유의하지 않으면 제외
pvalue_list2 = []
unsignificant_features = pvalue_df[pvalue_df.pvalue>alpha].feature.tolist()
for col in tqdm(unsignificant_features):
    d = train_df4[[col,'target']].rename(columns={col:'feature'})
    d['feature'] = log_offset(d['feature'])
    
    model = ols(f'feature ~ C(target)',data=d).fit()
    pvalue = anova_lm(model).values[0][-1]
    pvalue_list2.append([col,pvalue])
    
pvalue_df2 = pd.DataFrame(pvalue_list2,columns=['feature','pvalue'])\
    .sort_values('pvalue',ascending=False)

100%|██████████| 39/39 [00:12<00:00,  3.16it/s]


In [22]:
delete_features = pvalue_df2[pvalue_df2.pvalue> alpha].feature.tolist()
log_features    = pvalue_df2[pvalue_df2.pvalue<=alpha].feature.tolist()
print('> delete_features')
print('  - length : {}'.format(len(delete_features)))
print('  - feature_name : {}'.format(delete_features))
print('')
print('> log_features')
print('  - length : {}'.format(len(log_features)))
print('  - feature_name : {}'.format(log_features))

train_df5 = train_df4.copy()
train_df5.drop(delete_features,axis=1,inplace=True)
for col in log_features:
    train_df5[col] = log_offset(train_df5[col])
    
test_df5 = test_df4.copy()
test_df5.drop(delete_features,axis=1,inplace=True)
for col in log_features:
    test_df5[col] = log_offset(test_df5[col])

> delete_features
  - length : 33
  - feature_name : ['짙은안개*강수량(mm)', '강수여부*짙은안개', '연기/연무*풍향', '연기/연무*사건발생거리', '적설여부*짙은안개', '연기/연무', '짙은안개*적설량(cm)', '연기/연무*안개', '강설여부*짙은안개', '연기/연무*번개', '짙은안개*강설량(mm)', '짙은안개*풍향', '연기/연무*강수량(mm)', '강수여부*번개', '짙은안개', '짙은안개*안개', '번개*강수량(mm)', '짙은안개*사건발생거리', '강수여부*연기/연무', '풍향*사건발생거리', '주말여부*짙은안개', '번개*안개', '눈날림*짙은안개', '강수여부*안개', '안개', '안개*사건발생거리', '안개*풍향', '연기/연무*짙은안개', '안개*강수량(mm)', '풍향', '번개*사건발생거리', '번개*풍향', '번개']

> log_features
  - length : 6
  - feature_name : ['강수여부*사건발생거리', '강수량(mm)*사건발생거리', '강수여부*강수량(mm)', '강수량(mm)', '풍향*강수량(mm)', '풍향*적설량(cm)']


<br>

## Categorical Features

In [23]:
import scipy

In [24]:
# 참조 : https://stackoverflow.com/questions/25368284/fishers-exact-test-for-bigger-than-2-by-2-contingency-table
import math
def _dfs(mat, pos, r_sum, c_sum, p_0, p):

    (xx, yy) = pos
    (r, c) = (len(r_sum), len(c_sum))

    mat_new = []

    for i in range(len(mat)):
        temp = []
        for j in range(len(mat[0])):
            temp.append(mat[i][j])
        mat_new.append(temp)

    if xx == -1 and yy == -1:
        for i in range(r-1):
            temp = r_sum[i]
            for j in range(c-1):
                temp -= mat_new[i][j]
            mat_new[i][c-1] = temp
        for j in range(c-1):
            temp = c_sum[j]
            for i in range(r-1):
                temp -= mat_new[i][j]
            mat_new[r-1][j] = temp
        temp = r_sum[r-1]
        for j in range(c-1):
            temp -= mat_new[r-1][j]
        if temp <0:
            return
        mat_new[r-1][c-1] = temp

        p_1 = 1
        for x in r_sum:
            p_1 *= math.factorial(x)
        for y in c_sum:
            p_1 *= math.factorial(y)

        n = 0
        for x in r_sum:
            n += x
        p_1 /= math.factorial(n)

        for i in range(len(mat_new)):
            for j in range(len(mat_new[0])):
                p_1 /= math.factorial(mat_new[i][j])
        if p_1 <= p_0 + 0.00000001:
            #print(mat_new)
            #print(p_1)
            p[0] += p_1
    else:
        max_1 = r_sum[xx]
        max_2 = c_sum[yy]
        for j in range(c):
            max_1 -= mat_new[xx][j]
        for i in range(r):
            max_2 -= mat_new[i][yy]
        for k in range(min(max_1,max_2)+1):
            mat_new[xx][yy] = k
            if xx == r-2 and yy == c-2:
                pos_new = (-1, -1)
            elif xx == r-2:
                pos_new = (0, yy+1)
            else:
                pos_new = (xx+1, yy)
            _dfs(mat_new, pos_new, r_sum, c_sum, p_0, p)

def fisher_exact(table):

    row_sum = []
    col_sum = []

    for i in range(len(table)):
        temp = 0
        for j in range(len(table[0])):
            temp += table[i][j]
        row_sum.append(temp)

    for j in range(len(table[0])):
        temp = 0
        for i in range(len(table)):
            temp += table[i][j]
        col_sum.append(temp)

    mat = [[0] * len(col_sum)] * len(row_sum)
    pos = (0, 0)

    p_0 = 1

    for x in row_sum:
        p_0 *= math.factorial(x)
    for y in col_sum:
        p_0 *= math.factorial(y)

    n = 0
    for x in row_sum:
        n += x
    p_0 /= math.factorial(n)

    for i in range(len(table)):
        for j in range(len(table[0])):
            p_0 /= math.factorial(table[i][j])

    p = [0]
    _dfs(mat, pos, row_sum, col_sum, p_0, p)

    return p[0]

In [25]:
pvalue_list = []
for col in tqdm(cat_features):
    data = pd.crosstab(train_df5[col],train_df5[target_feature]).values
    stat, pvalue, dof, expected = scipy.stats.chi2_contingency(data)
    
    # 기대빈도가 5이하인 셀이 전체의 20%가 넘는 경우
    # -> fisher exact test가 필요함
    # (주의!) 기대빈도가 너무 높은 경우에는 math.factorial에서 에러발생함
    if np.where(data.flatten()<=5,1,0).sum() / len(data.flatten()) >= 0.2:
        print('{}: Fisher Exact Test is required'.format(col))
        pvalue = fisher_exact(data)
        
    pvalue_list.append([col,pvalue])

100%|██████████| 9/9 [00:00<00:00, 58.63it/s]


In [26]:
pvalue_df = pd.DataFrame(pvalue_list,columns=['feature','pvalue'])\
    .sort_values('pvalue',ascending=False)
#pvalue_df.round(4)

In [27]:
delete_features = pvalue_df[pvalue_df.pvalue>alpha].feature.tolist()
print('> delete_features')
print('  - length : {}'.format(len(delete_features)))
print('  - feature_name : {}'.format(delete_features))

train_df6 = train_df5.copy()
train_df6.drop(delete_features,axis=1,inplace=True)

test_df6 = test_df5.copy()
test_df6.drop(delete_features,axis=1,inplace=True)

> delete_features
  - length : 0
  - feature_name : []


In [28]:
target_feature = 'target'
cat_features = train_df6.columns[train_df6.dtypes==object]
cat_features = list(set(cat_features)-set([target_feature]))
num_features = [col for col in train_df6.columns if col not in cat_features+[target_feature]]

<br></br>

# Modeling

## CatBoostClassifier with K-Fold

In [29]:
from sklearn.model_selection import StratifiedKFold
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import f1_score

In [35]:
n_splits = 3
learning_rate = 0.03
iterations = 10000
early_stopping_rounds = 1000 # int(iterations/5)
random_state = 0

In [36]:
X = train_df6.drop(target_feature,axis=1)
y = train_df6[target_feature]

skf = StratifiedKFold(n_splits=n_splits,random_state=random_state,shuffle=True)
models = []

k=0
for train_idx, valid_idx in tqdm(skf.split(X,y),total=n_splits):
    k+=1
    X_train, y_train = X.iloc[train_idx], y.iloc[train_idx]
    X_valid, y_valid = X.iloc[valid_idx], y.iloc[valid_idx]
    
    train_dataset = Pool(X_train,y_train,cat_features=cat_features)
    valid_dataset = Pool(X_valid,y_valid,cat_features=cat_features)

    model = CatBoostClassifier(
        random_state=random_state,
        iterations=iterations,
        learning_rate=learning_rate,
        allow_writing_files=False,
    )
    model.fit(
        train_dataset,
        eval_set=valid_dataset,
        metric_period=int(iterations/5),
        early_stopping_rounds=early_stopping_rounds,
    )
    #model.save_model(f'./model_checkpoints/kfold_model_{k}.cbm')
    
    y_pred = model.predict(valid_dataset).flatten()
    y_true = y_valid.values
    score = f1_score(y_true=y_true,y_pred=y_pred,average='macro')
    
    print('K-Fold {}, Macro F1 Score: {:.4f}'.format(k,score))
    
    models.append(model)

  0%|          | 0/3 [00:00<?, ?it/s]Warning: Overfitting detector is active, thus evaluation metric is calculated on every iteration. 'metric_period' is ignored for evaluation metric.


0:	learn: 1.0914827	test: 1.0914479	best: 1.0914479 (0)	total: 147ms	remaining: 24m 28s
2000:	learn: 0.8976734	test: 0.9590405	best: 0.9590117 (1980)	total: 2m 29s	remaining: 9m 56s
Stopped by overfitting detector  (1000 iterations wait)

bestTest = 0.9589112724
bestIteration = 2174

Shrink model to first 2175 iterations.


 33%|███▎      | 1/3 [03:58<07:56, 238.47s/it]

K-Fold 1, Macro F1 Score: 0.5213


0:	learn: 1.0919514	test: 1.0916581	best: 1.0916581 (0)	total: 57.6ms	remaining: 9m 36s
2000:	learn: 0.9022572	test: 0.9466138	best: 0.9466017 (1998)	total: 2m 28s	remaining: 9m 55s
Stopped by overfitting detector  (1000 iterations wait)

bestTest = 0.9463709805
bestIteration = 2559

Shrink model to first 2560 iterations.


 67%|██████▋   | 2/3 [08:26<04:15, 255.71s/it]

K-Fold 2, Macro F1 Score: 0.5312


0:	learn: 1.0915125	test: 1.0916177	best: 1.0916177 (0)	total: 67.2ms	remaining: 11m 11s
2000:	learn: 0.8971727	test: 0.9608946	best: 0.9607746 (1809)	total: 2m 29s	remaining: 9m 56s
Stopped by overfitting detector  (1000 iterations wait)

bestTest = 0.9607746266
bestIteration = 1809

Shrink model to first 1810 iterations.


100%|██████████| 3/3 [11:57<00:00, 239.15s/it]

K-Fold 3, Macro F1 Score: 0.5224


In [38]:
import pickle
with open('./model_checkpoints/kfold_models.pkl', 'wb') as f:
	pickle.dump(models, f, protocol=pickle.HIGHEST_PROTOCOL)

<br>

## 참조 pycaret

In [39]:
# from pycaret import classification

# %%time
# classification.setup(data=train_df6,target='target',remove_outliers=True,verbose=True)
# best = classification.compare_models(n_select=5,fold=5)

<br></br>

# Inference

In [41]:
import pickle
with open('./model_checkpoints/kfold_models_3_featureselect_0.05.pkl', 'rb') as f:
    models = pickle.load(f)

<br>

## Train dataset

In [42]:
hard_voting_dict = {}
soft_voting_dict = {}
i=0
for model in tqdm(models):
    i+=1
    dataset = Pool(train_df4.drop(target_feature,axis=1),cat_features=cat_features)
    hard_voting_dict[str(i)] = model.predict(dataset).flatten()
    soft_voting_dict[str(i)] = model.predict_proba(dataset)

100%|██████████| 3/3 [00:02<00:00,  1.07it/s]


In [43]:
preds_hard_voting = pd.DataFrame(hard_voting_dict)\
    .progress_apply(lambda x: x.value_counts().sort_values(ascending=False).index.tolist()[0],axis=1)
preds_hard_voting = [int(float(x)) for x in preds_hard_voting]

proba_soft_voting = np.mean([proba for key,proba in soft_voting_dict.items()],axis=0)
preds_soft_voting = np.argmax(proba_soft_voting,axis=1)
preds_soft_voting = [int(float(x)) for x in preds_soft_voting]

100%|██████████| 84406/84406 [00:15<00:00, 5376.94it/s]


In [44]:
y_true = train_df4[target_feature].astype(float).astype(int)

score = f1_score(y_true=y_true,y_pred=preds_hard_voting,average='macro')
print('(1) Hard Voting : Macro F1 Score: {:.4f}'.format(score))
display(pd.crosstab(preds_hard_voting,y_true,rownames=['pred'],colnames=['true']))

score = f1_score(y_true=y_true,y_pred=preds_soft_voting,average='macro')
print('(1) Soft Voting : Macro F1 Score: {:.4f}'.format(score))
display(pd.crosstab(preds_soft_voting,y_true,rownames=['pred'],colnames=['true']))

(1) Hard Voting : Macro F1 Score: 0.5400


true,0,1,2
pred,,,
0,26134,8941,9871
1,4810,12370,3572
2,5509,4086,9113


(1) Soft Voting : Macro F1 Score: 0.5406


true,0,1,2
pred,,,
0,26103,8910,9876
1,4812,12400,3536
2,5538,4087,9144


<br>

## Test dataset

In [45]:
hard_voting_dict = {}
soft_voting_dict = {}
i=0
for model in tqdm(models):
    i+=1
    dataset = Pool(test_df4,cat_features=cat_features)
    hard_voting_dict[str(i)] = model.predict(dataset).flatten()
    soft_voting_dict[str(i)] = model.predict_proba(dataset)

100%|██████████| 3/3 [00:00<00:00,  5.02it/s]


In [46]:
preds_hard_voting = pd.DataFrame(hard_voting_dict)\
    .progress_apply(lambda x: x.value_counts().sort_values(ascending=False).index.tolist()[0],axis=1)
preds_hard_voting = [int(float(x)) for x in preds_hard_voting]

proba_soft_voting = np.mean([proba for key,proba in soft_voting_dict.items()],axis=0)
preds_soft_voting = np.argmax(proba_soft_voting,axis=1)
preds_soft_voting = [int(float(x)) for x in preds_soft_voting]

100%|██████████| 17289/17289 [00:03<00:00, 5261.35it/s]


In [47]:
# pred = preds_hard_voting
pred = preds_soft_voting

submit = pd.read_csv('./data/sample_submission.csv')
submit['TARGET'] = preds_hard_voting
submit['TARGET'] = submit['TARGET'].astype(float).astype(int)
submit.to_csv('./out/submission_kfold3_featureselect0.05.csv',index=False)